In [6]:
import pandas as pd
import numpy as np

In [34]:
def process_data(input_file, sleep_file, output_file, drop_columns=None):
    # Read input files
    if input_file.endswith('.xlsx'):
        df = pd.read_excel(input_file)
    elif input_file.endswith('.csv'):
        df = pd.read_csv(input_file)
    else:
        raise ValueError("Unsupported file format. Please provide data in Excel (.xlsx) or CSV (.csv) format.")
    
    # Drop unnecessary columns
    if drop_columns is not None:
        df.drop(columns=drop_columns, inplace=True)
    
    # Set 'value' to NaN and 'confidence' to 0 where confidence < 60
    df.loc[df['confidence'] < 60, ['value', 'confidence']] = np.nan, 0
    
    # Fill NaN in 'activity' with 'Chill'
    df['activity'].fillna('Chill', inplace=True)
    
    # Fill NaN in 'person' with the respective values in that column
    df['person'].fillna(df['person'].mode().iloc[0], inplace=True)
    
    # Group by 'unix_time' and aggregate columns
    agg_functions = {
        'value': 'mean', 
        'confidence': 'max',
        'person': 'first',
        'activity': 'first',
        'stress_lvl': 'first'
    }
    df_grouped = df.groupby('unix_time').agg(agg_functions).reset_index()
    
    # Add a 'datetime' column converted from 'unix_time' and shifted by an hour
    df_grouped['datetime'] = pd.to_datetime(df_grouped['unix_time'], unit='s') + pd.Timedelta(hours=1)
    
    # Read sleep data
    if sleep_file.endswith('.xlsx'):
        sleep_df = pd.read_excel(sleep_file)
    elif sleep_file.endswith('.csv'):
        sleep_df = pd.read_csv(sleep_file)
    else:
        raise ValueError("Unsupported file format for sleep data. Please provide data in Excel (.xlsx) or CSV (.csv) format.")
    
    # Initialize and fill 'sleep_duration' column
    df_grouped['sleep_duration'] = sleep_df.set_index('date')['sleep_duration'].reindex(df_grouped['datetime'].dt.date).fillna(0).tolist()
    
    # Rearrange columns
    df_grouped = df_grouped[['unix_time', 'datetime', 'person', 'value', 'confidence', 'activity', 'sleep_duration', 'stress_lvl']]
    
    # Write the processed DataFrame to a new Excel file
    df_grouped.to_excel(output_file, index=False)

In [35]:
process_data('raw_khang.xlsx', 'sleep_data_khang.xlsx', 'preprocessed_khang.xlsx', drop_columns=['name', 'date'])
process_data('raw_anh.xlsx', 'sleep_data_anh.xlsx', 'preprocessed_anh.xlsx', drop_columns=['name', 'date'])
process_data('raw_zahra.xlsx', 'sleep_data_zahra.xlsx', 'preprocessed_zahra.xlsx', drop_columns=['name', 'date'])